In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
from itertools import compress

import pprint

import re

import pandas as pd
from datetime import datetime

In [2]:
## iterable object

## what if we make a bunch of if-else statements?

cit0 = re.compile('чл.?\s?[0-9\-]*[\s,]\w*')
cit1 = re.compile('чл.\S?\s?[0-9]*[\s,]+ал.\S?[0-9\-]*[\s,]\w*')
cit2 = re.compile('чл.\S?\s?[0-9]*[\s,]+т\.\S?[0-9\-]*[\s,]\w*')
cit3 = re.compile('чл.?\s?[0-9]*[\s,]+ал.?\s?[0-9]*[\s,]+т.?\s?[0-9\-]*[\s,]\w*')

cit = (cit2, cit0, cit1, cit3)

In [3]:
# defining date range

dr = pd.date_range(start = "2021-03-14",end = datetime.today(), freq = 'W')
dr_str = dr.strftime("%#d.%#m.%Y")

In [4]:
#creating dictionary

dictionary = dict()

In [5]:
# loop for scraping x7 day iterations of the case law


for i in range(0,len(dr_str)-1):
    date1 = dr_str[i]
    date2 = dr_str[i+1]
    vks_url = "http://domino.vks.bg/bcap/scc/webdata.nsf/Acts?SearchView&query=(FIELD%20DocType%20=%2015)%20AND%20FIELD%20ud0_Date%20%3E=%20{}%20AND%20FIELD%20ud0_Date%20%3C=%20{}%20&SearchOrder=4&Start=1".format(date1, date2)
    uClient = urlopen(vks_url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "html.parser")
    findall = page_soup.findAll("a")
    for case in findall:
        link = case["href"]
        name = case.text
        uClient = urlopen(link)
        casepage_html = uClient.read()
        casepage_soup = soup(casepage_html, "html.parser")
        casetext = casepage_soup.text
        citations = [i.findall(casetext) for i in cit]
#        for i in range(1,3):
#            partof = [any(z in y for y in citations[i + 1]) == False for z in citations[i]]
#            citations[i] = list(compress(citations[i], partof))
#        partof = [any(z in y for y in citations[0]) == False for z in citations[1]]
#        citations[1] = list(compress(citations[1], partof))

        citations = [item for sublist in citations for item in sublist]    
        dictionary[name] = citations
